# Importing Libraries

In [1]:
import numpy as np
import torch
import time
import torch.utils.data
from torch.utils.data.dataset import Dataset
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import random
from typing import List

# Importing Dataset and Training settings
## Please DO NOT change this part

In [2]:
print(f'Pytorch version: {torch.__version__}')
print(f'cuda version: {torch.version.cuda}')
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

Pytorch version: 1.8.2
cuda version: 10.2
cuda


The ouput should be:
Pytorch version: 1.8.2
cuda version: 10.2
cuda

In [3]:
def START_seed():
    seed = 42
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

In [4]:
CLASSES = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

# Task5:One Network to Best Them All
### Using the previous observations and what you learned in the lecture please modify the following architecture to achieve best possible accuracy.
Examples of architectural changes to the baseline: adding more convolutional layers, more channels perconvolution layer, more fully connected layers, dropout, BN, data-augmentations, etc

In [5]:
# CLASSES = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
# Creating labels for dogcat output. Cat 3 -> 1, Dog: 5 -> 2, Everything else -> 0.

def convert_normal_to_dogcat(x: int):
    if x == 3:
        return 1
    elif x == 5:
        return 2
    return 0

def convert_dogcat_to_normal(x: int):
    if x == 1:
        return 3
    return 5

In [6]:
from PIL import Image

# We need to extend default dataset, because we have additional target for dogcat output.

class CIFAR10Extended(torchvision.datasets.CIFAR10):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.targets_dogcat = [convert_normal_to_dogcat(x) for x in self.targets]
    
    def __getitem__(self, index: int) -> List[torch.Tensor]:
        img = self.data[index]
        img = Image.fromarray(img)
        img = self.transform(img)
        return img, self.targets[index], self.targets_dogcat[index]

In [7]:
train_batch_size = 256
test_batch_size = 256

train_transform = transforms.Compose([
   transforms.RandomHorizontalFlip(),
   transforms.ToTensor(),
   transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

test_transform = transforms.Compose([
   transforms.ToTensor(),
   transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

train_set = CIFAR10Extended(root='./data', train=True, download=True, transform=train_transform)
test_set = CIFAR10Extended(root='./data', train=False, download=True, transform=test_transform)

train_loader = torch.utils.data.DataLoader(dataset=train_set, batch_size=train_batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_set, batch_size=test_batch_size, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [8]:
class Bottleneck(nn.Module):
    def __init__(self, in_planes: int, growth_rate: int) -> None:
        super(Bottleneck, self).__init__()
        self.bn1 = nn.BatchNorm2d(in_planes)
        self.conv1 = nn.Conv2d(in_planes, 4*growth_rate, kernel_size=1, bias=False)
        self.bn2 = nn.BatchNorm2d(4*growth_rate)
        self.conv2 = nn.Conv2d(4*growth_rate, growth_rate, kernel_size=3, padding=1, bias=False)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        out = self.conv1(F.relu(self.bn1(x)))
        out = self.conv2(F.relu(self.bn2(out)))
        out = torch.cat([out,x], 1)
        return out

class Transition(nn.Module):
    def __init__(self, in_planes: int, out_planes: int):
        super(Transition, self).__init__()
        self.bn = nn.BatchNorm2d(in_planes)
        self.conv = nn.Conv2d(in_planes, out_planes, kernel_size=1, bias=False)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        out = self.conv(F.relu(self.bn(x)))
        out = F.avg_pool2d(out, 2)
        return out

class DogCatHead(nn.Module):
    def __init__(self, num_features: int, num_classes: int) -> None:
        super().__init__()
        self.linear = nn.Linear(num_features, num_classes)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.linear(x)

class DenseNetDogCat(nn.Module):
    def __init__(self, block: nn.Module, nblocks: int, growth_rate: int=12, reduction: float=0.5, num_classes: int=10):
        START_seed()
        super(DenseNetDogCat, self).__init__()
        self.growth_rate = growth_rate

        num_planes = 2*growth_rate
        self.conv1 = nn.Conv2d(3, num_planes, kernel_size=3, padding=1, bias=False)

        self.dense1 = self._make_dense_layers(block, num_planes, nblocks[0])
        num_planes += nblocks[0]*growth_rate
        out_planes = int(np.floor(num_planes*reduction))
        self.trans1 = Transition(num_planes, out_planes)
        num_planes = out_planes

        self.dense2 = self._make_dense_layers(block, num_planes, nblocks[1])
        num_planes += nblocks[1] * growth_rate
        self.bn = nn.BatchNorm2d(num_planes)
        self.linear = nn.Linear(num_planes * 16, num_classes)
        self.dogcathead = DogCatHead(num_planes * 16, 3)

    def _make_dense_layers(self, block: nn.Module, in_planes: int, nblock: int):
        layers = []
        for i in range(nblock):
            layers.append(block(in_planes, self.growth_rate))
            in_planes += self.growth_rate
        return nn.Sequential(*layers)

    def forward(self, x: torch.Tensor):
        out = self.conv1(x)
        out = self.trans1(self.dense1(out))
        out = self.dense2(out)
        out = F.avg_pool2d(out, kernel_size=4)
        out = out.view(out.size(0), -1)                
        dogcat_out = self.dogcathead(out) 
        out = self.linear(out)
        return out, dogcat_out

def densenetdogcat_cifar():
    return DenseNetDogCat(Bottleneck, [4, 7], growth_rate=12)

In [9]:
from typing import Tuple

def train_dogcat(epoch: int, model: nn.Module, optimizer, lambda_: float) -> Tuple[torch.Tensor, torch.Tensor]:
    model.train()

    total_loss_per_epoch = 0
    correct = 0
    correct_dogcat = 0
    
    for batch_idx, (data, target, target_dogcat) in enumerate(train_loader):
        data, target, target_dogcat = data.to(device), target.to(device), target_dogcat.to(device)
        optimizer.zero_grad()
        output, output_dogcat = model(data)
        index_pred = torch.argmax(F.softmax(output, dim=1), dim=1)        
        
        loss = F.cross_entropy(output, target)
        loss_dogcat = F.cross_entropy(output_dogcat, target_dogcat, reduction="none") 
        # we don't do reduction for dogcat branch right away, 
        # because we want to assign zero loss to samples
        # which original class is neither dog nor cat in dogcat classification
        loss_dogcat[(index_pred != 3) & (index_pred != 5)] = 0 
        # now we can make reduction
        loss_dogcat = torch.mean(loss_dogcat)
        # calculate loss with two terms
        loss = (1 - lambda_) * loss + lambda_ * loss_dogcat
        loss.backward()
        
        optimizer.step()


def test_dogcat(model: nn.Module) -> Tuple[float, List, List]:
    model.eval()
    correct = 0
    
    for data, target, target_dogcat in test_loader:
        data, target = data.to(device), target.to(device)
        output, output_dogcat = model(data)
        
        pred = output.data.max(
            1, keepdim=True)[1]
        pred_dogcat = output_dogcat.data.max(
            1, keepdim=True)[1]
        
        for i, (p, p_dogcat) in enumerate(zip(pred, pred_dogcat)):
            # if normal branch predicted dog or cat
            # we look at the output of dogcat branch
            # and assign final prediction for that sample
            # based on the output of dogcat branch
            if p.item() == 3 or p.item() == 5:
                corrected_pred = convert_dogcat_to_normal(p_dogcat.item())
                pred[i] = corrected_pred
        
        correct += pred.eq(target.data.view_as(pred)).cpu().sum().item()

    accuracy = 100. * correct / len(test_loader.dataset)
    return accuracy

In [10]:
from typing import Tuple, Callable
from datetime import datetime

def train_and_test(epochs: int, model: nn.Module,
                   create_optimizer: Callable, lr: float, lambda_: float):
    date = datetime.now().strftime("%b%d_%H-%M-%S")
    START_seed()
    optimizer = create_optimizer()
    model.to(device)
    test_accs = []
    start = time.time()
    for epoch in range(1, epochs + 1):
        if epoch == 25:
            lr = lr / 10
            for param_group in optimizer.param_groups:
                param_group['lr'] = lr
        
        epoch_loss = train_dogcat(epoch, model, optimizer, lambda_)
        epoch_test_accuracy = test_dogcat(model)
        test_accs.append(epoch_test_accuracy)
        print(f"epoch={epoch} test_accuracy={epoch_test_accuracy}")    
    end = time.time()
    Total_time=end-start
    print('Total training and inference time is: {0}'.format(Total_time))
    start = time.time()
    accuracy = test_dogcat(model)
    end = time.time()
    Test_time=end-start
    print('Total inference time is: {0}'.format(Test_time))
    Training_time=Total_time-(Test_time*epochs)
    print('Total Training time is: {0}'.format(Training_time))
    max_test_acc = np.max(test_accs)
    max_test_acc_epoch = np.argmax(test_accs)
    print(f"Array of test_accuracy by epoch {test_accs}.")
    print(f"Maximum accuracy is {max_test_acc} and obtained on epoch {max_test_acc_epoch}.")
    print("Done")    


In [11]:
def create_optim() -> torch.optim:
    return optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=1e-3, nesterov=True)

model = densenetdogcat_cifar()
lr = 0.05
lambda_ = 0.1
print('Number of parameters: {0}'.format(sum(p.numel() for p in model.parameters())))
train_and_test(epochs=30, model=model, lr=lr, lambda_=lambda_, create_optimizer=create_optim)

Number of parameters: 120277
epoch=1 test_accuracy=43.02
epoch=2 test_accuracy=64.01
epoch=3 test_accuracy=66.71
epoch=4 test_accuracy=71.86
epoch=5 test_accuracy=67.64
epoch=6 test_accuracy=74.2
epoch=7 test_accuracy=78.96
epoch=8 test_accuracy=76.3
epoch=9 test_accuracy=76.12
epoch=10 test_accuracy=73.43
epoch=11 test_accuracy=77.67
epoch=12 test_accuracy=79.07
epoch=13 test_accuracy=81.83
epoch=14 test_accuracy=79.73
epoch=15 test_accuracy=79.0
epoch=16 test_accuracy=80.23
epoch=17 test_accuracy=77.17
epoch=18 test_accuracy=77.87
epoch=19 test_accuracy=81.82
epoch=20 test_accuracy=74.77
epoch=21 test_accuracy=79.96
epoch=22 test_accuracy=77.52
epoch=23 test_accuracy=80.92
epoch=24 test_accuracy=82.21
epoch=25 test_accuracy=88.17
epoch=26 test_accuracy=88.42
epoch=27 test_accuracy=88.48
epoch=28 test_accuracy=88.43
epoch=29 test_accuracy=88.74
epoch=30 test_accuracy=88.36
Total training and inference time is: 480.72797107696533
Total inference time is: 2.0288233757019043
Total Traini